<a href="https://colab.research.google.com/github/gongboogi/SchoolWork/blob/main/HonGong-mldl/hg_mldl_chapter05_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**hg_mldl chapter 05-3**
트리의 앙상블

---
### **정형 데이터와 비정형 데이터**
정형데이터: 어떤 구조로 되어있는 데이터
비정형 데이터: 텍스트(책), 디지털카메라 사진, 디지털 음악 등

**앙상블 학습**: 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘



###**랜덤 포레스트**
**랜덤 포레스트**: 결정 트리를 랜덤으로 만들어 숲을 만들고, 각 결정 트리의 예측을 사용하여 최종 예측을 만든다.

**부트스트랩 샘플**: 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식 훈련 세트와 크기가 같다.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data,target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# return_train_score 매개변수를 True로 지정하면 검증점수와 훈련세트에 대한 점수를 같이 반환(기본값=False)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) 

In [ ]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


-> 훈련 세트에 과대적합

In [ ]:
rf.fit(train_input, train_target) # 랜덤 포레스트  모델을 훈련세트에 훈련
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


**OOB(out of bag)샘플**: 부트스트랩 샘플에 포함되지 않고 남는 샘플. 검증 세트의 역할을 한다.

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


###**엑스트라 트리**

부트스트랩 샘플을 사용하지 않는다. 즉, 각 결정 트리를 만들 때 전체 훈련 세트를 사용

In [ ]:
# ExtraTreesClassifier: 사이킷런에서 제공하는 엑스트라 트리
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### **그레이디언트 부스팅**
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블.

결정 트리를 연속적으로 추가하여 손실함수를 최소화한다.

훈련속도가 느림. -> 개선한 앙상블: 히스토그램 기반 그레이디언트 부스팅

